In [37]:
import sys
sys.path.append("../qiskit")
sys.path.append("../pytorch")
from qiskit_library import *
# from mnist import *
import numpy as np
from random import randrange
import qiskit as qk
from qiskit import Aer
from qiskit import execute
import math
import sys
import random
import time
from tqdm import tqdm
import torch

def do_slp_via_th(input_ori,w_ori):
    p = input_ori
    d = 4*p*(1-p)
    e = (2*p-1)
    # e_sq = torch.tensor(1)
    w = w_ori
    
    sum_of_sq = (d+e.pow(2)).sum(-1)
    sum_of_sq = sum_of_sq.unsqueeze(-1)        
    sum_of_sq = sum_of_sq.expand(p.shape[0], w.shape[0])
            
    diag_p = torch.diag_embed(e)        
    
    p_w = torch.matmul(w,diag_p)
    # print(diag_p)
    # print(w)
    # print(p_w)
    # 
    z_p_w = torch.zeros_like(p_w)        
    shft_p_w = torch.cat((p_w, z_p_w), -1)
    
    sum_of_cross = torch.zeros_like(p_w)
    length = p.shape[1]    
    
    for shft in range(1,length):    
        sum_of_cross += shft_p_w[:,:,0:length]*shft_p_w[:,:,shft:length+shft]

    sum_of_cross = sum_of_cross.sum(-1)
            
    # print(sum_of_sq,sum_of_cross)
    return (sum_of_sq+2*sum_of_cross)/(length**2) 


print(do_slp_via_th(torch.tensor([[0.415,0.319,0.01,0.662]]),torch.tensor([[1.0,1.0,1.0,1.0]])))

def probility_perspective(input_ori):
    prob = input_ori[0]
    rand = []
    for p in prob:
        x = {}
        x[-1] = 1-p
        x[1] = p
        rand.append(x)
    
    # p_0 = 0
    # p_1 = 0
    # p_2 = 0
    # p_3 = 0
    # p_4 = 0
    # 
    sum_dist = {}
    for i0 in [-1,1]:
        for i1 in [-1,1]:
            for i2 in [-1,1]:
                for i3 in [-1,1]:
                    if i0+i1+i2+i3 not in sum_dist.keys():
                        sum_dist[i0+i1+i2+i3] = 0
                    sum_dist[i0+i1+i2+i3] += rand[0][i0]*rand[1][i1]*rand[2][i2]*rand[3][i3]                
    
    square_sum = 0 
    for k,v in sum_dist.items():
        # print(k,v)
        if k<=0:
            print(k/4,v)
            square_sum += (k/4)**2*v

    print(square_sum)
    # print(rand[0][1]*rand[1][1]*rand[2][1]*rand[3][1])        
    
            
    
    

probility_perspective(torch.tensor([[0.415,0.319,0.01,0.662]]))

tensor([[0.2616]])
-1.0 tensor(0.1333)
-0.5 tensor(0.4195)
0.0 tensor(0.3560)
tensor(0.2382)


In [ ]:

# Input = [-1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,  1.,  1.,  1.,  1.,
#          1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,  1.,  1.,  1.,  1.,
#          1.,  1.,  1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,  1.,  1.,
#          1.,  1.,  1.,  1.,  1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,
#          1.,  1.,  1.,  1.,  1., -1., -1., -1.]
# 
# 
# w_l1 = [         
#     [-1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,  1.,  1., -1.,
#           1.,  1., -1.,  1., -1.,  1., -1.,  1.,  1., -1., -1., -1., -1., -1.,
#          -1.,  1., -1., -1., -1.,  1.,  1.,  1., -1.,  1.,  1., -1., -1.,  1.,
#           1.,  1.,  1.,  1., -1., -1., -1.,  1., -1., -1., -1.,  1., -1.,  1.,
#          -1., -1.,  1., -1.,  1., -1., -1.,  1.],
#         [-1., -1.,  1., -1., -1., -1., -1., -1., -1.,  1., -1.,  1., -1., -1.,
#           1., -1.,  1., -1.,  1.,  1.,  1.,  1.,  1., -1., -1., -1., -1.,  1.,
#          -1.,  1.,  1.,  1.,  1., -1.,  1.,  1., -1.,  1.,  1.,  1.,  1.,  1.,
#           1.,  1.,  1.,  1., -1., -1., -1.,  1.,  1.,  1., -1.,  1.,  1.,  1.,
#          -1.,  1., -1.,  1.,  1., -1.,  1.,  1.],
#         [ 1., -1.,  1., -1.,  1., -1., -1., -1.,  1.,  1., -1.,  1.,  1.,  1.,
#           1.,  1., -1., -1.,  1., -1., -1.,  1.,  1.,  1.,  1., -1.,  1.,  1.,
#          -1., -1., -1., -1.,  1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,
#           1., -1., -1., -1., -1.,  1.,  1., -1., -1., -1., -1.,  1.,  1., -1.,
#           1., -1., -1., -1., -1., -1.,  1.,  1.],
#         [-1., -1.,  1.,  1., -1.,  1.,  1., -1.,  1., -1.,  1.,  1., -1., -1.,
#           1.,  1.,  1., -1.,  1.,  1., -1.,  1., -1.,  1., -1., -1., -1.,  1.,
#          -1., -1., -1.,  1.,  1.,  1., -1.,  1., -1., -1., -1., -1.,  1., -1.,
#          -1., -1.,  1.,  1., -1., -1., -1.,  1.,  1.,  1., -1., -1.,  1., -1.,
#          -1., -1., -1., -1., -1., -1., -1.,  1.]]
# 
# maxIndex = len(Input)
#         
# 
# 
# 
# q_en = qk.QuantumRegister(6,"encoded")
# q_out = qk.QuantumRegister(len(w_l1),"output")
# c = qk.ClassicalRegister(len(w_l1),"reg")
# aux = qk.QuantumRegister(4,"aux")
# 
# circuit = qk.QuantumCircuit(q_en, aux, q_out, c)
# 
# # SLP_8_encoding(circuit, q_io, q_en, input)
# # reset_qbits(q_io)
# 
# 
# for idx in range(len(w_l1)):
#     
#     for qbit in q_en:
#         circuit.h(qbit)
#     circuit.barrier()    
#     SLP_8_Uw(circuit, q_en, Input, aux)
#     circuit.barrier()
#     SLP_8_Uw(circuit, q_en, w_l1[idx], aux)
#     
#     circuit.barrier()
#     for qbit in q_en:
#         circuit.h(qbit)
#         circuit.x(qbit)
#     ccccccx(circuit,q_en,q_out[idx],aux)
#     # ccccx(circuit, q_en[0], q_en[1], q_en[2], q_en[3], q_out[idx], aux[0], aux[1])
#     circuit.barrier()
#     reset_qbits(circuit,q_en)

# 
# 
# # reset_qbits(q_en)
# circuit.barrier()
# 
# for idx in range(len(w_l1)):
#     circuit.swap(q_en[idx],q_out[idx])
#     
# circuit.barrier()
# 
# SLP_4_encoding(circuit,q_en[0:4],q_out[0:2],aux=aux)
#  
# print(circuit)
# 
# 
# 
# qc_shots = 100
# num_c_reg = len(w_l1)
# 
# print("="*50)
# print("Start simulation:")
# start = time.time()        
# iters = 1
# counts = simulate(circuit,qc_shots,iters,False)
# 
# end = time.time()
# qc_time = end - start
# 
# 
# print("From QC:",counts)
# print("Simulation elasped time:",qc_time)
#     
# def analyze(counts):
#     mycount = {}
#     for i in range(num_c_reg):
#         mycount[i] = 0
#     for k,v in counts.items():
#         bits = len(k) 
#         for i in range(bits):            
#             if k[bits-1-i] == "1":
#                 if i in mycount.keys():
#                     mycount[i] += v
#                 else:
#                     mycount[i] = v
#     return mycount,bits
# 
# # for k,v in counts[0].items():
# #     print(k,v)
# (mycount,bits) = analyze(counts[0])
# 
# for b in range(bits):
#     print (b,float(mycount[b])/qc_shots)
#     
# 
# 
# print("="*100)
# import torch
# print("Theoretic Analysis")
# 
# 
# test_input = (torch.tensor([Input])+1)/2
# test_weight = torch.tensor(w_l1)
# print(do_slp_via_th(test_input,test_weight))






In [ ]:

# INPUTs
import torch
from torch import tensor

IFM = tensor([[0.1098, 0.1765, 0.0510, 0.0000, 0.1333, 0.4784, 0.3608, 0.0000, 0.0000,
         0.1922, 0.4549, 0.0980, 0.0000, 0.0706, 0.2824, 0.1137]])
W1_0 = tensor([[-1.,  1., -1.,  1.,  1., -1.,  1., -1.,  1., -1., -1., -1., -1.,  1.,
          1., -1.]])
W1_1 = tensor([[-1., -1., -1., -1., -1.,  1., -1., -1., -1.,  1.,  1., -1., -1., -1.,
         -1., -1.]])
W1_2 = tensor([[ 1.,  1.,  1.,  1.,  1.,  1., -1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,
          1.,  1.]])
W1_3 = tensor([[ 1.,  1., -1., -1.,  1.,  1., -1., -1.,  1.,  1., -1., -1., -1.,  1.,
         -1.,  1.]])

W1 = W1_3

OFM1 = tensor([[0.0316, 0.3768, 0.3218, 0.0273]])

OFM1_QC = tensor([[0.0349, 0.3748, 0.3283, 0.0280]])

W2_0 =tensor([[ 1.,  1., -1.,  1.]])
W2_1 =tensor([[-1.,  1., -1., -1.]])

W2 = W2_1

OFM2 = tensor([[0.3238, 0.3756]])


test_L1 = False

if test_L1:
    input = torch.tensor(IFM[0])*2-1
    
    q_in = qk.QuantumRegister(16,"io")
    q_enc = qk.QuantumRegister(4,"encoder")
    q_out = qk.QuantumRegister(len(W1),"output")
    c = qk.ClassicalRegister(len(W1),"reg")
    aux = qk.QuantumRegister(4,"aux")

    maxIndex = len(input)
    circuit = qk.QuantumCircuit(q_in, q_enc, q_out, aux, c)
    
    for idx in range(len(W1)):
        SLP_16_encoding(circuit, q_in, q_enc, input, aux)    
        SLP_16_Uw(circuit,q_enc,W1[idx], aux)
        circuit.barrier()
    
        for qbit in q_enc[0:4]:
            circuit.h(qbit)
            circuit.x(qbit)
        ccccx(circuit, q_enc[0], q_enc[1], q_enc[2], q_enc[3], q_out[idx], aux[0], aux[1])
        circuit.barrier()
        
        # reset_qbits(circuit,q_in)
        # reset_qbits(circuit,q_enc)
    
    for idx in range(len(W1)):        
        circuit.measure(q_out[idx],c[idx])

        
else:
    input = torch.tensor(OFM1_QC[0])*2-1
    
    q_in = qk.QuantumRegister(4,"io")
    q_enc = qk.QuantumRegister(2,"encoder")
    q_out = qk.QuantumRegister(len(W2),"output")
    c = qk.ClassicalRegister(len(W2),"reg")
    aux = qk.QuantumRegister(4,"aux")

    maxIndex = len(input)
    circuit = qk.QuantumCircuit(q_in, q_enc, q_out, aux, c)

    for idx in range(len(W2)):
        SLP_4_encoding(circuit, q_in, q_enc, input, aux)
        
        SLP_4_Uw(circuit,q_enc,W2[idx], aux)
        circuit.barrier()
    
        for qbit in q_enc[0:2]:
            circuit.h(qbit)
            circuit.x(qbit)
        circuit.ccx(q_enc[0], q_enc[1], q_out[idx])
        circuit.barrier()
        
        # reset_qbits(circuit,q_in)
        # reset_qbits(circuit,q_enc)
    
    for idx in range(len(W1)):        
        circuit.measure(q_out[idx],c[idx])


print(circuit)

In [ ]:

qc_shots = 10000
num_c_reg = len(W1)

print("="*50)
print("Start simulation:")
start = time.time()        
iters = 1
counts = simulate(circuit,qc_shots,iters,False)

end = time.time()
qc_time = end - start


print("From QC:",counts)
print("Simulation elasped time:",qc_time)
    
def analyze(counts):
    mycount = {}
    for i in range(num_c_reg):
        mycount[i] = 0
    for k,v in counts.items():
        bits = len(k) 
        for i in range(bits):            
            if k[bits-1-i] == "1":
                if i in mycount.keys():
                    mycount[i] += v
                else:
                    mycount[i] = v
    return mycount,bits

# for k,v in counts[0].items():
#     print(k,v)
(mycount,bits) = analyze(counts[0])

for b in range(bits):
    print (b,float(mycount[b])/qc_shots)
    

In [ ]:
print("="*100)
import torch
print("Theoretic Analysis")



test_input = IFM
print(test_input)
print(do_slp_via_th(test_input,W1))

### Comparison Reulst:

#### IFM

0.1098, 0.1765, 0.0510, 0.0000,
 
0.1333, 0.4784, 0.3608, 0.0000,

0.0000, 0.1922, 0.4549, 0.0980,
 
0.0000, 0.0706, 0.2824, 0.1137

#### Layer 1:
Theoretic_Result = [0.0316, 0.3768, 0.3218, 0.0273]

Quantum_Resuts = [0.0349, 0.3748, 0.3283, 0.0280] # Shots: 10^4

#### Layer 2:


Theoretic_Result = [0.3238, 0.3756]

Quantum_Resuts = [0.3140, 0.3797] # Shots: 10^4, Using Theoretic_Results in Layer 1

Quantum_Resuts = [0.3215, 0.3725] # Shots: 10^4, Using Quantum_Resuts in Layer 1